<a href="https://colab.research.google.com/github/CargoCultScientist/ny-opendata/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
from requests.auth import HTTPBasicAuth
import requests

api_id = userdata.get("NY_API_KEY_ID")
api_secret = userdata.get("NY_API_KEY_SECRET")

DOMAIN = "data.ny.gov"
DATASET_ID = "n9v6-gdp6"
url = f"https://{DOMAIN}/resource/{DATASET_ID}.json?$limit=10"

auth = HTTPBasicAuth(api_id, api_secret)
r = requests.get(url, auth=auth)
r.raise_for_status()
for i, row in enumerate(r.json(), start=1):
    print(f"{i}. {row.get('current_entity_name')} – {row.get('initial_dos_filing_date')}")


1. BUTCHY'S WINE & SPIRITS, INC. – 2013-06-27T00:00:00.000
2. GRACE SEAFOOD CORP. – 2011-03-24T00:00:00.000
3. KV CATALYST IM, LLC – 2021-07-15T00:00:00.000
4. MADISON TAX GROUP INC. – 2013-02-13T00:00:00.000
5. REVERSALS INC. – 2012-07-02T00:00:00.000
6. WILD APACHE PRODUCTIONS INC. – 1990-09-18T00:00:00.000
7. YAWGER BROOK BAKES, INC. – 1989-02-28T00:00:00.000
8. 33-23 STEUBEN AVENUE CORP. – 2004-07-26T00:00:00.000
9. CHRISTOPHER W. EDWARDS, P.C. – 2002-08-30T00:00:00.000
10. FARWAY MARINA INC. – 1991-07-22T00:00:00.000


In [1]:
# Task 2 — Active Corporations (NY Open Data)
# A) Print the 10 most recent additions (sorted by Initial DOS Filing Date)
# B) Download the full JSON snapshot to disk
#
# Requirements: store your credentials in Colab Secrets as:
#   NY_API_KEY_ID        → the API Key ID
#   NY_API_KEY_SECRET    → the API Key Secret

from google.colab import userdata
from requests.auth import HTTPBasicAuth
import requests, json
from datetime import datetime
import pandas as pd # Import pandas for table output

# ---- Secrets and config ------------------------------------------------------
API_ID = userdata.get("NY_API_KEY_ID")
API_SECRET = userdata.get("NY_API_KEY_SECRET")
if not API_ID or not API_SECRET:
    raise RuntimeError("Missing secrets. Add NY_API_KEY_ID and NY_API_KEY_SECRET in Runtime → Secrets.")

auth = HTTPBasicAuth(API_ID, API_SECRET)

DOMAIN = "data.ny.gov"
DATASET_ID = "n9v6-gdp6"  # Active Corporations: Beginning 1800

FIELDS = [
    "dos_id",
    "current_entity_name",
    "initial_dos_filing_date",
    "county",
    "jurisdiction",
    "entity_type",
]

# ---- A) Fetch 10 newest by filing date (dataset stores date as text MM/DD/YYYY) -----------
# Use SoQL to parse the text date to a real timestamp on the server, then order DESC.
resource_url = f"https://{DOMAIN}/resource/{DATASET_ID}.json"
params = {
    "$select": ",".join(FIELDS),
    "$order": "initial_dos_filing_date DESC",
    "$limit": 10,
}

resp = requests.get(resource_url, params=params, auth=auth, timeout=60)
resp.raise_for_status()
rows = resp.json()

# Create a pandas DataFrame for better display
df = pd.DataFrame(rows)

# Print results as a table
print("10 newest by Initial DOS Filing Date:")
display(df)

# Save the 10 records to a small JSON file (handy for submission or checks)
with open("/content/latest_10_active_corporations.json", "w", encoding="utf-8") as f:
    json.dump(rows, f, ensure_ascii=False, indent=2)
print("Saved: /content/latest_10_active_corporations.json")

# ---- B) Download full dataset snapshot ---------------------------------------
# This endpoint returns the entire dataset export as JSON.
snapshot_url = f"https://{DOMAIN}/api/views/{DATASET_ID}/rows.json?accessType=DOWNLOAD"
out_path = "/content/active_corporations_latest.json"

with requests.get(snapshot_url, auth=auth, stream=True, timeout=300) as r2:
    r2.raise_for_status()
    with open(out_path, "wb") as f:
        for chunk in r2.iter_content(chunk_size=1 << 20):
            if chunk:
                f.write(chunk)

print(f"Saved full dataset snapshot to: {out_path}")

10 newest by Initial DOS Filing Date:


,dos_id,current_entity_name,initial_dos_filing_date,county,jurisdiction,entity_type
0,7743016,104 SAG NO MORE LLC,2025-10-25T00:00:00.000,Essex,New York,DOMESTIC LIMITED LIABILITY COMPANY
1,7742952,1780 SK LLC,2025-10-25T00:00:00.000,Westchester,New York,DOMESTIC LIMITED LIABILITY COMPANY
2,7742958,1908 DELI INC.,2025-10-25T00:00:00.000,Nassau,New York,DOMESTIC BUSINESS CORPORATION
3,7742945,1914 HUTCHINSON PKWY LLC,2025-10-25T00:00:00.000,Westchester,New York,DOMESTIC LIMITED LIABILITY COMPANY
4,7742966,2514 DELI INC.,2025-10-25T00:00:00.000,Nassau,New York,DOMESTIC BUSINESS CORPORATION
5,7742980,312 STRATEGIES LLC,2025-10-25T00:00:00.000,New York,New York,DOMESTIC LIMITED LIABILITY COMPANY
6,7742949,44-46 ELMWOOD LLC,2025-10-25T00:00:00.000,Westchester,New York,DOMESTIC LIMITED LIABILITY COMPANY
7,7742938,44TH STREET FILMS LLC,2025-10-25T00:00:00.000,Queens,New York,DOMESTIC LIMITED LIABILITY COMPANY
8,7743026,88 JZC CHINA WOK LLC,2025-10-25T00:00:00.000,Kings,New York,DOMESTIC LIMITED LIABILITY COMPANY
9,7743040,A.M SATTAR CORPORATION,2025-10-25T00:00:00.000,Suffolk,New York,DOMESTIC BUSINESS CORPORATION


Saved: /content/latest_10_active_corporations.json
Saved full dataset snapshot to: /content/active_corporations_latest.json


In [ ]:
# Task 2 — Active Corporations (NY Open Data, n9v6-gdp6)
# A) Print the 10 most recent additions by Initial DOS Filing Date
# B) Download the full JSON snapshot to disk
#
# Colab Secrets required:
#   NY_API_KEY_ID        → API Key ID
#   NY_API_KEY_SECRET    → API Key Secret

from google.colab import userdata
from requests.auth import HTTPBasicAuth
import requests, json
from datetime import datetime
from typing import List, Dict, Any, Optional

# ----------------------------- Config & Auth ---------------------------------
API_ID = userdata.get("NY_API_KEY_ID")
API_SECRET = userdata.get("NY_API_KEY_SECRET")
if not API_ID or not API_SECRET:
    raise RuntimeError("Missing secrets. Add NY_API_KEY_ID and NY_API_KEY_SECRET in Runtime → Secrets.")

auth = HTTPBasicAuth(API_ID, API_SECRET)

DOMAIN = "data.ny.gov"
DATASET_ID = "n9v6-gdp6"
RESOURCE_URL = f"https://{DOMAIN}/resource/{DATASET_ID}.json"

FIELDS = [
    "dos_id",
    "current_entity_name",
    "initial_dos_filing_date",
    "county",
    "jurisdiction",
    "entity_type",
]

# ----------------------------- Helpers ---------------------------------------
def parse_date_any(s: str) -> Optional[datetime]:
    """Handle both ISO timestamps and 'MM/DD/YYYY' text."""
    if not s:
        return None
    s = s.strip()
    # Try ISO-like formats first
    for fmt in ("%Y-%m-%d", "%Y-%m-%dT%H:%M:%S.%f", "%Y-%m-%dT%H:%M:%S"):
        try:
            return datetime.strptime(s, fmt)
        except ValueError:
            pass
    # Try US style text date
    for fmt in ("%m/%d/%Y",):
        try:
            return datetime.strptime(s, fmt)
        except ValueError:
            pass
    return None

def normalize_date_str(s: str) -> str:
    dt = parse_date_any(s) if s else None
    return dt.date().isoformat() if dt else (s or "")

def client_sorted_latest_10(rows: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """Sort rows by parsed filing date desc and return top 10."""
    with_dates = [(parse_date_any(r.get("initial_dos_filing_date", "")), r) for r in rows]
    with_dates = [(d, r) for d, r in with_dates if d is not None]
    with_dates.sort(key=lambda x: x[0], reverse=True)
    latest = [r for _, r in with_dates[:10]]
    for r in latest:
        r["initial_dos_filing_date"] = normalize_date_str(r.get("initial_dos_filing_date"))
    return latest

# ----------------------------- Strategy --------------------------------------
# 1) Fast path: ask the API for the newest records by internal row id (:id).
#    Then sort by the filing date client-side for correctness.
#    Grabbing a small window (e.g., 5000 rows) is plenty to capture the latest 10 by date.
# 2) If that fails, fall back to a smaller window (e.g., 1000).
# 3) Print the latest 10 and also save them to a JSON file.
# 4) Download the full snapshot JSON to disk.

def fetch_recent_window(window_size: int = 5000) -> List[Dict[str, Any]]:
    params = {
        "$select": ",".join(FIELDS),
        "$where": "initial_dos_filing_date is not null",
        "$order": ":id DESC",   # newest rows first by ingestion order
        "$limit": window_size,
    }
    r = requests.get(RESOURCE_URL, params=params, auth=auth, timeout=120)
    r.raise_for_status()
    return r.json()

def fetch_latest_10() -> List[Dict[str, Any]]:
    # Try a 5k window first; if anything goes wrong or <10 parsed, fall back to 1k.
    for size in (5000, 2000, 1000):
        try:
            rows = fetch_recent_window(size)
            latest10 = client_sorted_latest_10(rows)
            if len(latest10) == 10:
                return latest10
        except requests.HTTPError as e:
            # Try the next smaller window
            if size == 1000:
                raise SystemExit(f"API error {e.response.status_code}: {e.response.text}") from e
    raise RuntimeError("Could not obtain 10 recent records with parseable dates.")

# ----------------------------- Run: Part A -----------------------------------
latest10 = fetch_latest_10()
for i, r in enumerate(latest10, start=1):
    print(
        f"{i:>2}. DOS ID: {r.get('dos_id','')} | "
        f"Name: {r.get('current_entity_name','')} | "
        f"Initial DOS Filing Date: {r.get('initial_dos_filing_date','')} | "
        f"County: {r.get('county','')} | "
        f"Jurisdiction: {r.get('jurisdiction','')} | "
        f"Type: {r.get('entity_type','')}"
    )

print(f"\nTotal shown: {len(latest10)} newest by Initial DOS Filing Date.")

# Save the 10 records to a small JSON file for submission/use
out_10 = "/content/latest_10_active_corporations.json"
with open(out_10, "w", encoding="utf-8") as f:
    json.dump(latest10, f, ensure_ascii=False, indent=2)
print(f"Saved: {out_10}")

# ----------------------------- Run: Part B -----------------------------------
# Full dataset snapshot (entire export) to disk
snapshot_url = f"https://{DOMAIN}/api/views/{DATASET_ID}/rows.json?accessType=DOWNLOAD"
out_snapshot = "/content/active_corporations_latest.json"

with requests.get(snapshot_url, auth=auth, stream=True, timeout=600) as r2:
    r2.raise_for_status()
    with open(out_snapshot, "wb") as f:
        for chunk in r2.iter_content(chunk_size=1 << 20):
            if chunk:
                f.write(chunk)

print(f"Saved full dataset snapshot to: {out_snapshot}")


 1. DOS ID: 7737028 | Name: ZORN UNITED INC. | Initial DOS Filing Date: 2025-10-18 | County: Kings | Jurisdiction: New York | Type: DOMESTIC BUSINESS CORPORATION
 2. DOS ID: 7736285 | Name: ZW EQUIPMENT RENTAL LLC | Initial DOS Filing Date: 2025-10-17 | County: Richmond | Jurisdiction: New York | Type: DOMESTIC LIMITED LIABILITY COMPANY
 3. DOS ID: 7735953 | Name: ZONE GO LLC | Initial DOS Filing Date: 2025-10-16 | County: New York | Jurisdiction: New York | Type: DOMESTIC LIMITED LIABILITY COMPANY
 4. DOS ID: 7734721 | Name: ZORA LOGISTIC LLC | Initial DOS Filing Date: 2025-10-15 | County: Queens | Jurisdiction: New York | Type: DOMESTIC LIMITED LIABILITY COMPANY
 5. DOS ID: 7733714 | Name: ZUBAS WELLNESS LLC | Initial DOS Filing Date: 2025-10-14 | County: Rockland | Jurisdiction: New York | Type: DOMESTIC LIMITED LIABILITY COMPANY
 6. DOS ID: 7733655 | Name: ZOFTWARE DESIGN LLC | Initial DOS Filing Date: 2025-10-14 | County: Oneida | Jurisdiction: New York | Type: DOMESTIC LIMITED LI

KeyboardInterrupt: 